In [3]:
!pip install selenium webdriver-manager

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import time
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium.webdriver import ActionChains

#1. 주유소 웹사이트 불러오기
def loadOilWeb(driver):
    
    # 화면 최대 크기 설정 
    driver.maximize_window()
    driver.get(url)
    #모든 창이 로딩 될떄까지 기다려 줌 
    time.sleep(5)
    #팝업창 화면 전환 후 닫아주기
    driver.switch_to.window(driver.window_handles[-1])
    driver.close()
    time.sleep(3)
    # 접근 페이지 다시 요청
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(url)

    url="https://www.opinet.co.kr/searRgSelect.do"
    driver=webdriver.Chrome('../driver/chromedriver.exe')
    loadOilWeb(driver)
    #서울 선택
    sido_select=driver.find_element_by_id("SIDO_NM0")
    sido_select.send_keys('서울')
    time.sleep(2)
    # 서울시 구 리스트 추출
    sigungu_select=driver.find_element(By.ID,'SIGUNGU_NM0')
    sigungu_options=sigungu_select.find_elements(By.TAG_NAME,'option')
    sigungu_list= [ option.get_attribute('value') for option in sigungu_options] # 서울시 구 리스트 
    print(sigungu_list)

#Beautiful Soup을 이용하여 주유소 데이터 추출 


def getInfoDetail(driver, seoul_gu):
    
    seoul_oil_station=[]
    gu_list=[] # 구 
    car_wash_list=[] # 세차장
    charging_list=[] #충전소
    maintenance_list=[] #경정비
    convenience_list=[] #편의점
    sel24_list=[] #24시 영업
    self_list=[] #셀프 
    name_list = [] #주유소 명
    brand_list=[] #브랜드
    address_list=[] #주소
    gasolin_price_list=[] # 가솔린가격
    diesel_price_list=[] #경유가격

    car_wash='' 
    charging='' 
    maintenance='' 
    convenience='' 
    sel24='' 
    self='' 
    
    req=driver.page_source
    soup=BeautifulSoup(req,'html.parser')        
    result_oil_info=soup.select_one('.result_gis #os_price1 #body1')
    time.sleep(2) 

    # 해당 구의 모든 주유소: 주유소명, 브랜드, 주소 
    oil_detail_list=result_oil_info.select('.rlist')
    
    #구 리스트
    gu_list=[seoul_gu for i in range(len(oil_detail_list)) ]
    
    
    for idx, detail in tqdm(enumerate(oil_detail_list)):
        
        # 셀프 여부 
        if '셀프' in detail.text.strip():
            self='Y'
        else:
            self='N'
            
        self_list.append(self)

        # inner info 
        #Beautiful Soup을 이용하여 주유소 데이터 추출
        # 로딩되기 전 데이터를 불러올 경우 에러 발생 

        rlist=driver.find_element(By.CSS_SELECTOR,f'#body1 > tr:nth-child({idx+1}) > td.rlist > a')
        
        time.sleep(4) 

        # 각 주유소를 클릭해서 inner info 접근 
        rlist.click()
        time.sleep(4) 

        req=driver.page_source
        soup=BeautifulSoup(req,'html.parser')
        inner_station_info=soup.select_one('.ollehmap-info #os_dtail_info')

        name = inner_station_info.select_one('#os_nm').text #주유소 명
        brand=inner_station_info.select_one('#poll_div_nm').text #브랜드
        address=inner_station_info.select_one('#rd_addr').text #주소
        gasolin_price=inner_station_info.select_one('#b027_p').text # 가솔린가격
        diesel_price=inner_station_info.select_one('#d047_p').text #경유가격
    

        # 각 리스트에 추가 
        name_list.append(name)
        brand_list.append(brand)
        address_list.append(address)
        gasolin_price_list.append(gasolin_price)
        diesel_price_list.append(diesel_price)
        
        
        # 부가정보 데이터 
        # 세차장, 충전소, 경정비, 편의점, 24시 영업
        service_info=inner_station_info.select_one('.service')
        #세차장 
        car_wash_img=service_info.select_one('#cwsh_yn').get('src')
        if 'off' in car_wash_img:
            car_wash='N'
        else:
            car_wash='Y'

        # 충전소
        charging_img=service_info.select_one('#lpg_yn').get('src')
        if 'off' in charging_img:
            charging='N'
        else:
            charging='Y'
        # 경정비
        maintenance_img=service_info.select_one('#maint_yn').get('src')
        if 'off' in maintenance_img:
            maintenance='N'
        else:
            maintenance='Y'
        #편의점
        convenience_img=service_info.select_one('#cvs_yn').get('src')
        if 'off' in convenience_img:
            convenience='N'
        else:
            convenience='Y'
        # 24시 영업
        sel24_img=service_info.select_one('#sel24_yn').get('src')
        if 'off' in convenience_img:
            sel24='N'
        else:
            sel24='Y'

        # 각 리스트에 추가  
        car_wash_list.append(car_wash)
        charging_list.append(charging)
        maintenance_list.append(maintenance)
        convenience_list.append(convenience)
        sel24_list.append(sel24)

        time.sleep(2)

    
    # 주유소 데이터 통합 
    data={
        '주유소명' : name_list,
        '주소' : address_list,
        '브랜드' : brand_list,
        '휘발유 가격' : gasolin_price_list,
        '경유 가격' : diesel_price_list,
        '셀프 여부' : self_list,
        '세차장 여부' : car_wash_list,
        '충전소 여부' : charging_list,
        '경정비 여부' : maintenance_list,
        '편의점 여부' : convenience_list,
        '24시간 운영 여부' : sel24_list,
        '구' : gu_list
    }
    
    print(data)
    

    return data

seoul_oil_station_dict={}

def getStationInfo(driver,sigungu_list):
            
    for gu in tqdm(sigungu_list[1:]):
        # 서울시 구 선택
        sigungu_select=driver.find_element(By.ID,'SIGUNGU_NM0')
        sigungu_select.send_keys(gu)
        time.sleep(2)
        # 군/구 버튼 클릭 
        sigungu_btn = driver.find_element(By.CSS_SELECTOR,'#searRgSelect')
        sigungu_btn.click()
        time.sleep(3)
        # 주유소 딕셔너리에 구별 주유소 데이터 추가 
        seoul_oil_station_dict[gu]=getInfoDetail(driver,gu)
        time.sleep(2)

        
        
    seoul_oil_station_dict
    
    driver.quit()

        

        
    getStationInfo(driver, sigungu_list) #주유소 세부정보 가져오기
    import pandas as pd
    import numpy as np

    car_wash_dict_list=[] # 세차장
    charging_dict_list=[] #충전소
    maintenance_dict_list=[] #경정비
    convenience_dict_list=[] #편의점
    sel24_dict_list=[] #24시 영업
    self_dict_list=[] #셀프 
    name_dict_list = [] #주유소 명
    brand_dict_list=[] #브랜드
    address_dict_list=[] #주소
    gasolin_dict_price_list=[] # 가솔린가격
    diesel_dict_price_list=[] #경유가격
    gu_dict_list=[] # 구 

    for key in seoul_oil_station_dict:
    
        car_wash_dict_list+=[val for val in seoul_oil_station_dict[key]['세차장 여부']]
        charging_dict_list+=[val for val in seoul_oil_station_dict[key]['충전소 여부']]
        maintenance_dict_list+=[val for val in seoul_oil_station_dict[key]['경정비 여부']]
        convenience_dict_list+=[val for val in seoul_oil_station_dict[key]['편의점 여부']]
        sel24_dict_list+=[val for val in seoul_oil_station_dict[key]['24시간 운영 여부']]
        self_dict_list+=[val for val in seoul_oil_station_dict[key]['셀프 여부']]
        name_dict_list+=[val for val in seoul_oil_station_dict[key]['주유소명']]
        brand_dict_list+=[val for val in seoul_oil_station_dict[key]['브랜드']]
        address_dict_list+=[val for val in seoul_oil_station_dict[key]['주소']]
        gasolin_dict_price_list+=[val for val in seoul_oil_station_dict[key]['휘발유 가격']]
        diesel_dict_price_list+=[val for val in seoul_oil_station_dict[key]['경유 가격']]
        gu_dict_list+=[val for val in seoul_oil_station_dict[key]['구']]

    oil_station_df.to_csv('c://seoul_oil_station_info_data.csv',sep=',',encoding='utf-8')